<a href="https://colab.research.google.com/github/ritzdevp/team-samosa-tvqa/blob/main/Multimodal_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ritzdevp/team-samosa-tvqa

Cloning into 'team-samosa-tvqa'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 253 (delta 1), reused 0 (delta 0), pack-reused 251
Receiving objects: 100% (253/253), 1.28 MiB | 11.90 MiB/s, done.
Resolving deltas: 100% (137/137), done.


In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1m8bC4lefQsP2tRhMLAaiy0AVuBXZtegc' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1m8bC4lefQsP2tRhMLAaiy0AVuBXZtegc" -O tvqa_imagenet_resnet101_pool5_hq.tar.gz && rm -rf /tmp/cookies.txt

--2022-04-26 17:37:28--  https://docs.google.com/uc?export=download&confirm=t&id=1m8bC4lefQsP2tRhMLAaiy0AVuBXZtegc
Resolving docs.google.com (docs.google.com)... 108.177.126.101, 108.177.126.139, 108.177.126.100, ...
Connecting to docs.google.com (docs.google.com)|108.177.126.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-6g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0j84m115ul5n7ct8c6o70clau6rltr4b/1650994575000/07218467666063224277/*/1m8bC4lefQsP2tRhMLAaiy0AVuBXZtegc?e=download [following]
--2022-04-26 17:37:28--  https://doc-0k-6g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0j84m115ul5n7ct8c6o70clau6rltr4b/1650994575000/07218467666063224277/*/1m8bC4lefQsP2tRhMLAaiy0AVuBXZtegc?e=download
Resolving doc-0k-6g-docs.googleusercontent.com (doc-0k-6g-docs.googleusercontent.com)... 172.217.218.132, 2a00:1450:4013:c08::84
Connecting to doc-0k-6g-docs.googleusercontent.com (doc-

In [59]:
! tar xzf /content/tvqa_imagenet_resnet101_pool5_hq.tar.gz

^C


In [1]:
!pip install -q transformers datasets

In [2]:
from transformers import BertTokenizer, BertModel
import torch
import transformers
from transformers import BertTokenizer, VisualBertForQuestionAnswering, VisualBertModel

In [3]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",  output_hidden_states = True)
bert_model = VisualBertModel.from_pretrained("uclanlp/visualbert-vqa-coco-pre",  output_hidden_states = True)


Some weights of the model checkpoint at uclanlp/visualbert-vqa-coco-pre were not used when initializing VisualBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing VisualBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisualBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
% cd /content/team-samosa-tvqa/baseline_repo

/content/team-samosa-tvqa/baseline_repo


In [ ]:
! chmod 777 /content/team-samosa-tvqa/baseline_repo/setup.sh

In [ ]:
! /content/team-samosa-tvqa/baseline_repo/setup.sh

In [5]:
#debugging
import h5py
import torch
from torch.nn.utils.rnn import pad_sequence
from CONSTANTS import RESNET_FEATURES

h5driver=None
vid_feat_path=RESNET_FEATURES
vid_h5 = h5py.File(vid_feat_path, "r", driver=h5driver)

def read_resnet_feats(video_names, stride=8):
    video_resnet_feat = []
    for video in video_names:
        video_resnet_feat.append(torch.tensor(vid_h5[video][::stride, :], device="cuda"))

    video_resnet_feat =  pad_sequence(video_resnet_feat, batch_first=True)
    
    return video_resnet_feat

In [6]:
class TVQA_Multimodal(torch.nn.Module):

    def __init__(self):

        super(TVQA_Multimodal, self).__init__()

        self.bert_tokenizer = bert_tokenizer
        self.bert_model = bert_model.to("cuda")

        self.proj = torch.nn.Sequential(torch.nn.Linear(768, 256),
                                        torch.nn.GELU(),
                                        torch.nn.Linear(256, 64),
                                        torch.nn.GELU(),
                                        torch.nn.Linear(64, 1)).to("cuda")

    def _freeze_bert_weights(self, requires_grad_layers = ['encoder.layer.11', 
                                                           'pooler.dense.weight',
                                                           'pooler.dense.bias']):
        for name, p in self.bert_model.named_parameters():
            req_grad = False
            for layer_name in requires_grad_layers:
                if layer_name in name:
                    req_grad = True
                    break

            p.requires_grad = req_grad
        
        return True

    def merge_q_ans(self, que, ans):
        merged = []
        for i in range(len(que)):
          curr = que[i]+ ' [SEP] ' + ans[i]
          merged.append(curr)
        return merged

    def bert_forward(self, question, ans, subt_text, video_names):

        merge_q_ans = self.merge_q_ans(question, ans)

        inputs       = bert_tokenizer(subt_text, 
                                      merge_q_ans,  
                                      padding="max_length",
                                      truncation=True,
                                      return_token_type_ids=True,
                                      return_attention_mask=True,
                                      add_special_tokens=True,
                                      return_tensors="pt")
        
        
        visual_embeds = read_resnet_feats(video_names)

        visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long).to("cuda")
        visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.float).to("cuda")
        inputs.update(
            {
                "visual_embeds": visual_embeds,
                "visual_token_type_ids": visual_token_type_ids,
                "visual_attention_mask": visual_attention_mask,
            }
        )
        inputs = inputs.to("cuda")

        output = bert_model(**inputs)

        hidden_states = output.last_hidden_state
        cls_tokens = hidden_states[:,0,:]

        return cls_tokens

    def forward(self, question, subt_text, a0, a1, a2, a3, a4, video_names):

        bert_a0 = self.bert_forward(question=question,
                                    ans=a0,
                                    subt_text=subt_text,
                                    video_names=video_names)        
        score_a0 = self.proj(bert_a0)

        bert_a1 = self.bert_forward(question=question,
                                    ans=a1,
                                    subt_text=subt_text,
                                    video_names=video_names)
        score_a1 = self.proj(bert_a1)

        bert_a2 = self.bert_forward(question=question,
                                    ans=a2,
                                    subt_text=subt_text,
                                    video_names=video_names)
        score_a2 = self.proj(bert_a2)

        bert_a3 = self.bert_forward(question=question,
                                    ans=a3,
                                    subt_text=subt_text,
                                    video_names=video_names)
        
        score_a3 = self.proj(bert_a3)

        bert_a4 = self.bert_forward(question=question,
                                    ans=a4,
                                    subt_text=subt_text,
                                    video_names=video_names)
        
        score_a4 = self.proj(bert_a4)

        # print(" score_a4 ", score_a4.shape)


        logits = torch.cat((score_a0, score_a1, score_a2, score_a3, score_a4), dim=1)

        # print(" logits ", logits.shape)


        return logits


In [7]:
from CONSTANTS import  BASE_PATH
import json

In [8]:


class TVQAPlus(torch.utils.data.Dataset):
    def __init__(self, dataset='train'):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        TRAIN_DICT_JSON = BASE_PATH + "/tvqa_plus/tvqa_plus_train.json"
        VAL_DICT_JSON = BASE_PATH + '/tvqa_plus/tvqa_plus_val.json'
        SUBTITLES_DICT_JSON = BASE_PATH + '/tvqa_plus/tvqa_plus_subtitles.json'

        self.dataset = dataset

        train_dict = {}
        val_dict = {}
        subtitles_dict = {}

        with open(TRAIN_DICT_JSON) as f:
          train_dict = json.load(f)

        with open(VAL_DICT_JSON) as f:
          val_dict = json.load(f)

        with open(SUBTITLES_DICT_JSON) as f:
          self.subtitles_dict = json.load(f)

        self.target_dict = {}
        if self.dataset == 'train':
          self.target_dict = train_dict
        elif self.dataset == 'val':
          self.target_dict = val_dict
        else:
          raise Exception(f"Invalid dataset passed {self.dataset}")
      
    def __len__(self):
        return len(self.target_dict)
        
    def __getitem__(self, i):

      question = self.target_dict[i]['q']
    
      a0       = self.target_dict[i]['a0']
      a1       = self.target_dict[i]['a1']
      a2       = self.target_dict[i]['a2']
      a3       = self.target_dict[i]['a3']
      a4       = self.target_dict[i]['a4']

      answer_idx = int(self.target_dict[i]['answer_idx'])
      
      video_name = self.target_dict[i]['vid_name']
      subtitle = self.subtitles_dict[video_name]
      subt_text = subtitle['sub_text']

      return question, subt_text, a0, a1, a2, a3, a4, video_name, answer_idx

In [9]:
train_dataset = TVQAPlus('train')
test_dataset = TVQAPlus("val")

In [10]:
batch_size=20
dev_batch_size=12

In [11]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=dev_batch_size, shuffle=False)

In [12]:
def merge(que, ans):
  merged = []
  for i in range(len(que)):
    curr = que[i]+ ' [SEP] ' + ans[i]
    merged.append(curr)
  return merged

In [13]:
tvqa_model = TVQA_Multimodal()
# tvqa_model._freeze_bert_weights()

print(tvqa_model.named_parameters())

for name, p in tvqa_model.named_parameters():
    print("name", name, "p", p.requires_grad)

<generator object Module.named_parameters at 0x7f91a24b2850>
name bert_model.embeddings.word_embeddings.weight p True
name bert_model.embeddings.position_embeddings.weight p True
name bert_model.embeddings.token_type_embeddings.weight p True
name bert_model.embeddings.LayerNorm.weight p True
name bert_model.embeddings.LayerNorm.bias p True
name bert_model.embeddings.visual_token_type_embeddings.weight p True
name bert_model.embeddings.visual_position_embeddings.weight p True
name bert_model.embeddings.visual_projection.weight p True
name bert_model.embeddings.visual_projection.bias p True
name bert_model.encoder.layer.0.attention.self.query.weight p True
name bert_model.encoder.layer.0.attention.self.query.bias p True
name bert_model.encoder.layer.0.attention.self.key.weight p True
name bert_model.encoder.layer.0.attention.self.key.bias p True
name bert_model.encoder.layer.0.attention.self.value.weight p True
name bert_model.encoder.layer.0.attention.self.value.bias p True
name bert_mo

In [14]:
def val_acc(model, dev_loader, batch_size_dev):
  model.eval()
  num_correct = 0
  for batch_idx, ( question, subt_text, a0, a1, a2, a3, a4, video_names, answer_idx) in enumerate(test_loader):
    answer_idx = answer_idx.to("cuda")
    with torch.no_grad():
      # # IF MODEL does not TAKES VIDEO INPUT

      logits = model.forward(question, subt_text, a0, a1, a2, a3, a4, video_names)

      
    num_correct += int((torch.argmax(logits, axis=1) == answer_idx).sum())
    acc = 100 * num_correct / ((batch_idx + 1) * batch_size_dev)

  dev_acc = 100 * num_correct / (len(dev_loader) * batch_size_dev)

  model.train()
  return dev_acc

In [15]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

optimizer = optim.Adam(tvqa_model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [ ]:
torch.cuda.empty_cache()

# Set the verbosity level as follows:

transformers.logging.set_verbosity_error()


# bert_model.cuda()

num_correct = 0

epoch = 0
best_dev_acc = 0
loss_epoch = 0

while epoch < 100:
  
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    tvqa_model.train()

    for batch_idx, ( question, subt_text, a0, a1, a2, a3, a4, video_names, answer_idx) in enumerate(train_loader):

        logits = tvqa_model.forward(question, subt_text, a0, a1, a2, a3, a4, video_names)
        answer_idx = answer_idx.to("cuda")
        loss = criterion(logits, answer_idx)
        num_correct += int((torch.argmax(logits, axis=1) == answer_idx).sum())

        loss.backward()
        optimizer.step()
        loss_epoch += float(loss)
        optimizer.zero_grad()

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / ((batch_idx + 1) * batch_size)),
            loss="{:.04f}".format(float(loss_epoch / (batch_idx + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        batch_bar.update() # Update tqdm bar

        # break

    batch_bar.close() # You need this to close the tqdm bar

    train_acc = 100 * num_correct / (len(train_loader) * batch_size)
    dev_acc = val_acc(tvqa_model, test_loader, dev_batch_size)

    print(f'Epoch {epoch} Loss {loss_epoch} train_acc {train_acc}, devacc {dev_acc}')
        


In [ ]:
tvqa_model = TVQA_Multimodal()
tvqa_model._freeze_bert_weights()
# print(tvqa_model)
# tvqa_model._free

for name, p in tvqa_model.bert_model.named_parameters():
    print("name", name, "p", p.shape, "p.requires_grad", p.requires_grad)